# Objet : Test opendata des données de qualité de l'air

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- interêt d'utiliser les outils Pandas
- à compléter

## Usages possibles 
- à compléter

## Autres points
- chargement sur MongoDB à tester
- Outil de requète à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
#os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from observation import Ilist, Iindex
from copy import copy
import pandas as pd

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/air/data_lcsqa/'

In [2]:

data = []
t0 = time()
annee = 2022
mois = 1
jour = 1
for i in range(6):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2 : \n', len(data2), '\n', list(data2), '\n', time()-t0)


data2 : 
 295807 
 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 
 22.650558471679688


----
## initialisation de l'objet Ilist
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [3]:
t0=time()
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
idxs2.setvar('valeur')
idxs3 = Ilist(idxs2)
print(idxs3 == idxs2)

idxs (len, lenlidx, sumcodec) :  295807 22 5147 2.810992479324341
True


## formats de base

In [4]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='full')
fullsize = len(js)
print('fullsize', len(js), time()-t0)

fullsize 86202592 16.426617860794067


In [5]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 295807 65.32035875320435
controle égalité : True 2.554198741912842


In [6]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='nokeys')
minsize = len(js)
print('minsize', len(js), time()-t0)

t0=time()
js = idxs2.to_obj(encoded=True, encode_format='cbor', modecodec='nokeys')
print('mincborsize', len(js), time()-t0)


minsize 1694137 0.8960316181182861
mincborsize 2339853 0.8734214305877686


----
## format default
- 

In [7]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='default')
defaultsize = len(js)
print('defaultsize : ', len(js), time()-t0, '\n')
print('indicator default : ', idxs2.indicator(fullsize, defaultsize), '\n')
t0=time()
pprint(idxs2.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl']), width=120)
pprint(idxs2.indexinfos(keys=['linkrate']))
print('analyse : ', time()-t0)

defaultsize :  23275238 1.731583595275879 

indicator default :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 3.574, 'unicity level': 0.0, 'object lightness': 0.27, 'gain': 0.73} 

[{'lencodec': 144, 'name': 'Date de début', 'num': 0, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 144, 'name': 'Date de fin', 'num': 1, 'parent': 0, 'typecoupl': 'coupled'},
 {'lencodec': 18, 'name': 'Organisme', 'num': 2, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'code zas', 'num': 3, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'Zas', 'num': 4, 'parent': 3, 'typecoupl': 'coupled'},
 {'lencodec': 533, 'name': 'code site', 'num': 5, 'parent': 5, 'typecoupl': 'crossed'},
 {'lencodec': 533, 'name': 'nom site', 'num': 6, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 5, 'name': "type d'implantation", 'num': 7, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 9, 'name': 'Polluant', 'num': 8, 'parent': 13, 'typecoup

In [8]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 295807 4.506702899932861
True
controle égalité : True 2.314021110534668


----
## Format optimisé
- 

In [9]:
t0=time()
js = idxs2.to_obj(modecodec='optimize', encoded=True)
optimizesize = len(js)
print('optimizesize : ', optimizesize, time()-t0, '\n')
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize), '\n')
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='optimize', encode_format='cbor')
cborsize = len(js)
print('cborsize : ', cborsize, time()-t0, '\n')
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

optimizesize :  13405403 2.1804146766662598 

indicator optimize :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 2.057, 'unicity level': 0.0, 'object lightness': 0.155, 'gain': 0.844} 

cborsize :  6912425 1.8778035640716553 

indicator cbor :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 1.059, 'unicity level': 0.0, 'object lightness': 0.08, 'gain': 0.92}


In [10]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 295807 4.601051330566406
True
controle égalité : True 2.3537251949310303


----
## Format BD
- 

In [11]:
t0=time()
js = idxs2.to_obj(modecodec='dict', encoded=True)
dictsize = len(js)
print('dictsize : ', dictsize, time()-t0, '\n')
print('indicator dict : ', idxs2.indicator(fullsize, dictsize), '\n')

dictsize :  49784605 6.336913585662842 

indicator dict :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 7.649, 'unicity level': 0.0, 'object lightness': 0.577, 'gain': 0.422} 



In [12]:
t0 = time()
test = idxs2.lindex[0].to_dict_obj()
print(time()-t0)

0.2120039463043213


In [13]:
print(len(idxs2.lindex[0].codec), len(idxs2.lindex[0].keys))

144 295807


In [14]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 295807 6.778273344039917
True
controle égalité : True 2.6191530227661133
